In [0]:
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession
import requests as req
import datetime as dt
import pyspark.pandas as pd

In [0]:
def extract_data(url, acess_code, searched_word, initial_data = None, final_data = None):
    """
    Realiza extração dos dados da API da NEWSAPI com base dos parâmetros recebidos

    Parameters
    ----------
    url: string
        String do método get da API.

    acess_code: string
        String com a chave usada para coletar os dados da API.

    searched_word: string
        String com a palavra a ser buscada na notícia.
        
    initial_data: date
        Data inicial a ser buscada, no formato 'YYYY-MM-DD'.

    final_data: date
        Data final a ser buscada, 'YYYY-MM-DD'.

    Returns
    -------
    response_df: float
        DataFrame com o retorno dos dados da API

    """
    if initial_data == None:
        initial_data = dt.datetime.now().date() - dt.timedelta(days=1)

    if final_data == None:
        final_data = dt.datetime.now().date() - dt.timedelta(days=1)

    parameters = {
        'q': searched_word,
        'apiKey': acess_code,
        'sortBy': 'publishedAt',
        'from': initial_data,
        'to': final_data
    }

    response = req.get(url, params = parameters)
    response_dic = response.json()

    if response_dic['status'] != 'ok':
        messageerror = response_dic['message']
        raise Exception(messageerror)
        
    dict_res = {}
    #percorre todas as noticias encontradas no resultado da chamada para criar uma chave única
    for item in response_dic['articles']:
        chave = item['url'] + item['publishedAt']
        dict_res[chave] = {
            "source": item['source']
            ,"author": item['author']
            ,"title": item['title']
            ,"description": item['description']
            ,"url":  item['url']
            ,"urlToImage": item['urlToImage']
            ,"publishedAt": item['publishedAt']
            ,"content": item['content']
        }

    response_df = pd.DataFrame.from_dict(dict_res, orient='index').reset_index().rename(columns={"index": "id"})
    return response_df


In [0]:
def load_data(new_data, path, file_name, key):
    """
    Realiza a leitura do arquivo com os dados já salvo e adiciona os novos dados

    Parameters
    ----------
    new_data: DataFrame
        Dataframe contendo os dados coletados da API.

    path: string
        Diretório dentro do dbfs onde os dados serão salvos.

    file_name: string
        Nome do arquivo em formato parquet onde os dados serão salvos

    key: string
        Nome do campo que identifica o registro como único
    

    Returns
    -------
    None

    """
    #caso já exista o arquivo transformado, segue direto com a concatenação e com a carga do resultado final
    try: 
        complete_path = path + file_name
        arquivo = dbutils.fs.ls(complete_path)
        df_res = pd.read_parquet(complete_path)
        df_res = pd.concat([df_res, new_data])
        df_res.to_parquet(complete_path)

         #elimina os registros duplicados baseado na chave passada antes de armazenar o resultado final
        df_res = df_res.sort_values(by="publishedAt")
        df_res = df_res.drop_duplicates(subset=key, keep='last')

        #armazena o resultado final
        df_res.to_parquet(complete_path)
    #caso o arquivo transformando ainda não exista, quer dizer que é o primeiro processo do pipeline e é preciso criar o arquivo destino
    except Exception as e: 
        if 'java.io.FileNotFoundException' in str(e):
            print("Arquivo não encontrado, primeiro processamento")
            new_data.to_parquet(complete_path)

    print("resultado carregado com sucesso")
    

In [0]:
def check_folder(path):
    """
    Realiza verificação de existência do diretório onde os dados serão salvos

    Parameters
    ----------
    path: string
        Diretório dentro do dbfs onde os dados serão salvos.

    Returns
    -------
    None

    """
    try:#Verifica se o diretório passado existe. Caso não, cria
        dbutils.fs.ls(path)
    except Exception as e:
        if 'java.io.FileNotFoundException' in str(e):
            print("Diretório não existente. Criando diretório.")
            dbutils.fs.mkdirs(path) 

In [0]:
#define o diretório de qual será o arquivo parquet final que irá armazenar os dados carregados
path = "/dbfs/data_newsapi/teste/"
file_name = "data_newsapi.parquet"

In [0]:
url = "https://newsapi.org/v2/everything?"
acess_code = "ed08e4049379498ebde784dee9d1ede8"
searched_word = "cancer OR DNA OR genetic"
key = "id"

response_df = extract_data(url, acess_code, searched_word)
check_folder(path)
load_data(response_df,path,file_name,key)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:467: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  A field of type StructType expects a pandas.DataFrame, but got: <class 'pandas.core.series.Series'>
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


Arquivo não encontrado, primeiro processamento
resultado carregado com sucesso
